# Building an image classifier

We're going to build an archaeological image classifier using Tensorflow, a library of python packages designed to perform machine learning tasks. We are going to train a model to recognize different kinds of Roman pottery.

The data we're going to use comes from [Potsherd: Atlas of Roman Pottery](http://potsherd.net/atlas/potsherd). I've already downloaded the dataset for you, and there's a block of code below designed to retrieve it for you. It's not the full database; rather, I just put together a small set of ten or so different types of pottery, just to keep things manageable.

In [ ]:
import urllib.request

# Retrieving the resource located at the URL and storing it in a zip file
url = "https://github.com/shawngraham/hist3000/blob/master/static/data/data.zip?raw=true"
urllib.request.urlretrieve(url, "data.zip")

!unzip data.zip

In [ ]:
# now get the tensorflow package
!pip install tensorflow

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Set up matplotlib for better plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 8)

# Check TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Load the Data!

In [ ]:
data_dir = Path("data/ware")

# List all classes (subdirectories)
class_names = [d.name for d in data_dir.iterdir() if d.is_dir()]
print(f"📁 Found {len(class_names)} classes: {class_names}")
    
# Count images in each class
for class_name in class_names:
    class_path = data_dir / class_name
    image_count = len(list(class_path.glob("*")))
    print(f"   {class_name}: {image_count} images")

# Split the data into training and testing datasets

We typically split data so that 80% of it is used for training, and 20% is held back for testing and validation.

First call (subset="training"):

+ validation_split=0.2 → "Split the data: 80% training, 20% validation"
+ subset="training" → "Give me the 80% training portion"
+ Result: train_ds gets 80% of the data


Second call (subset="validation"):

+ validation_split=0.2 → "Split the data: 80% training, 20% validation" (same split!)
+ subset="validation" → "Give me the 20% validation portion"
+ Result: val_ds gets 20% of the data

In [20]:
# Image parameters
IMG_HEIGHT = 224  # Standard size for most pre-trained models
IMG_WIDTH = 224
BATCH_SIZE = 32

# Create training dataset (80% of data)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,  
    subset="training",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical'  # One-hot encoding for multiple classes
)

# Create validation dataset (20% of data)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Get class names from the dataset
class_names = train_ds.class_names
num_classes = len(class_names)
print(f"📊 Number of classes: {num_classes}")
print(f"📊 Class names: {class_names}")

Found 56 files belonging to 12 classes.
Using 12 files for training.
Found 56 files belonging to 12 classes.
Using 11 files for validation.
📊 Number of classes: 12
📊 Class names: ['AHGW', 'AOMO', 'ARGO', 'B4', 'BB1', 'BB2', 'C189', 'CGBL', 'CGCC', 'CGGL', 'CGMW', 'CGTS']


# Visualize Some Training Images

In [ ]:
# Let's look at some of our training images
plt.figure(figsize=(12, 8))

# Take one batch of images and labels
for images, labels in train_ds.take(1):
    # Show first 9 images
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        
        # Convert image to displayable format
        img = images[i].numpy().astype("uint8")
        plt.imshow(img)
        
        # Get the class name from the one-hot encoded label
        class_idx = np.argmax(labels[i])
        plt.title(f"Class: {class_names[class_idx]}")
        plt.axis("off")

plt.suptitle("Sample Training Images", fontsize=16)
plt.tight_layout()
plt.show()

# Data Pre-Processing

In [ ]:
# Normalize pixel values to [0, 1] range
# This helps the neural network train more effectively
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Apply normalization to both datasets
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Configure datasets for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("✅ Data preprocessing complete!")
print("📋 Images are now normalized to [0, 1] range")

# Build a neural network model
On the basis of a pre-trained one

In [ ]:
# We'll use transfer learning with MobileNetV2
# This is a pre-trained model that already knows about images

# Load the pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,  # Don't include the final classification layer
    weights='imagenet'  # Use weights trained on ImageNet
)

# Freeze the base model (don't train it)
base_model.trainable = False

# Build our custom model
model = tf.keras.Sequential([
    base_model,                                    # Pre-trained feature extractor
    tf.keras.layers.GlobalAveragePooling2D(),     # Convert features to 1D
    tf.keras.layers.Dropout(0.2),                 # Prevent overfitting
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Final classification layer
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
print("🧠 Neural Network Architecture:")
model.summary()

# Now we train it!

In [ ]:
# Set up training parameters
EPOCHS = 10  # Number of times to go through the entire dataset; more generally but not always gives better results

print(f"🚀 Starting training for {EPOCHS} epochs...")
print("This might take a few minutes...")

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1  # Show progress
)

print("✅ Training complete!")

# Now we assess how well the model has been trained


In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot training & validation accuracy
ax1.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
ax1.set_title('Model Accuracy Over Time')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True)

# Plot training & validation loss
ax2.plot(history.history['loss'], label='Training Loss', marker='o')
ax2.plot(history.history['val_loss'], label='Validation Loss', marker='s')
ax2.set_title('Model Loss Over Time')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# Print final results
final_acc = history.history['val_accuracy'][-1]
print(f"🎯 Final Validation Accuracy: {final_acc:.2%}")

# Make a prediction: given this image, what kind of ware is it?

In [ ]:
# Incidentally, you don't want to be retraining a model everytime you want to do some classification.
# So we can save it like this:

model_path = "image_classifier_model.h5"
model.save(model_path)
print(f"💾 Model saved as: {model_path}")

# And then, for the functions below, if you were resuming work, you could load the model with:
# loaded_model = tf.keras.models.load_model(model_path)

In [ ]:
# Function to make predictions on a batch of images
def predict_and_display(dataset, num_images=6):
    """Display images with their predicted and actual labels"""
    
    plt.figure(figsize=(15, 10))
    
    # Get one batch of images
    for images, labels in dataset.take(1):
        # Make predictions
        predictions = model.predict(images)
        
        # Display the first num_images
        for i in range(min(num_images, len(images))):
            plt.subplot(2, 3, i + 1)
            
            # Show image
            img = images[i].numpy()
            plt.imshow(img)
            
            # Get actual and predicted classes
            actual_class = class_names[np.argmax(labels[i])]
            predicted_class = class_names[np.argmax(predictions[i])]
            confidence = np.max(predictions[i])
            
            # Set title color based on correctness
            color = 'green' if actual_class == predicted_class else 'red'
            
            plt.title(f'Actual: {actual_class}\n'
                     f'Predicted: {predicted_class}\n'
                     f'Confidence: {confidence:.2%}', 
                     color=color)
            plt.axis('off')
    
    plt.suptitle('Model Predictions on Validation Images', fontsize=16)
    plt.tight_layout()
    plt.show()

# Show predictions
predict_and_display(val_ds)

In [ ]:
# Create a function to predict a single image
def predict_single_image(image_path):
    """Predict the class of a single image"""
    
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(
        image_path, 
        target_size=(IMG_HEIGHT, IMG_WIDTH)
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize
    
    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = np.max(predictions[0])
    
    # Display results
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title(f'Predicted: {predicted_class}\nConfidence: {confidence:.2%}')
    plt.axis('off')
    plt.show()
    
    # Show confidence for all classes
    print("🔍 Confidence scores for all classes:")
    for i, class_name in enumerate(class_names):
        print(f"   {class_name}: {predictions[0][i]:.2%}")

# Go out and find a picture of something Roman, and predict!

Go to the [Potshered: Atlas of Roman Pottery](https://potsherd.net/atlas/potsherd) and find some pottery that interests you. You can then right-click on an image and select 'copy url to image'. Then we can feed it to our model to see what the model predicts using this command:

```
predict_single_image("path/to/your/test/image.jpg")
```

In [ ]:
import urllib.request
from PIL import Image

# Retrieving the resource located at the URL
# and storing it in the file name a.png
url = "https://potsherd.net/atlas/gallery2/ITTS/ITTS-stamp.jpg"
urllib.request.urlretrieve(url, "ITTS-stamp.jpg")

# Opening the image and displaying it (to confirm its presence)
img = Image.open(r"ITTS-stamp.jpg")
img.show()

predict_single_image("ITTS-stamp.jpg")

# this example image won't be too good a result, since we didn't train the model on any Terra Sigillata! 
# But go look at the ware type in the atlas - maybe the failure of the model also teaches us something.
# And maybe the ware that you select will be correctly identified!

## Go Further!

Find more classes of archaeological materials and build your own classifier. The key element is finding *enough* images - more is better - and then organizing them where each folder name is the name of the category. 

Then, you re-run this notebook but point the initial path to your own data/categories/category1 etc folder, ie the line where it says ```data_dir = Path("data/ware")``` you change.